In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as ticker
from fitter import Fitter
import random
import dill

In [80]:
geburtenRate = dill.load(open('../Daten/zf_geburtenrate/distributions_nach_sprache.pick', 'rb'))
todesRaten = dill.load(open('../Daten/zf_todesrate/distributions_nach_sprache.pick', 'rb'))
einwanderung = dill.load(open('../Daten/zf_einwanderung/distributions_nach_sprache.pick', 'rb'))
auswanderung = dill.load(open('../Daten/zf_auswanderung/distributions_nach_sprache.pick', 'rb'))

print(geburtenRate)

def generateDistribution(distr_type_params,n=10000):
    key = list(distr_type_params.keys())[0]
    params = distr_type_params[key]
    obj = getattr(stats, key)
   
    if type(params) is dict:
        params = list(params.values())

    dist_vals = obj.rvs(size=n,*params)
 
    return dist_vals

distirbutionDict = {}

for sprache in list(geburtenRate.keys()):
    lang_dict = distirbutionDict.get(sprache, {})

    lang_dict["birth"] = generateDistribution(geburtenRate[sprache])
    lang_dict["death"] = generateDistribution(todesRaten[sprache])
    lang_dict["einwanderung"] = generateDistribution(einwanderung[sprache])
    lang_dict["auswanderung"] = generateDistribution(auswanderung[sprache])
    lang_dict["zufallsFaktor"] = generateDistribution({"norm":{"loc": 1, "alpha":0.1}})


    distirbutionDict[sprache] = lang_dict

print(distirbutionDict)

{'Deutsch': {'dgamma': (1.6250865911513537, 42019.57212323723, 1695.253331348888)}, 'Französisch': {'rdist': (1.0770523924411726, 14629.493262931104, 2281.5390805676802)}, 'Italienisch': {'gompertz': (1.3686532288608229, 4555.656031622897, 1220.3977902035622)}, 'Andere': {'laplace_asymmetric': (0.33890103771291025, 15554.695619860337, 609.28975118311)}}
{'Deutsch': {'birth': array([40356.43992111, 39739.66766896, 36487.25482198, ...,
       43180.14408213, 38966.60266102, 38357.97818391]), 'death': array([33033.71879543, 31393.61017375, 30544.30791775, ...,
       30693.11265202, 27802.05137573, 31081.85733536]), 'einwanderung': array([31545.695684  , 46668.07803226, 44620.57292887, ...,
       39150.10947637, 32084.70011788, 48643.84678745]), 'auswanderung': array([41331.55662656, 46086.05734795, 41339.4703194 , ...,
       48531.98064164, 44486.67983765, 32802.45177141]), 'zufallsFaktor': array([0.8577577 , 0.93665738, 1.02090503, ..., 1.05843616, 1.03236058,
       1.09237941])}, 'F

In [81]:
startValue = 7000000

def simulation(distirbutionDict,startValue,years=50,n=10):
    runs = []
    for i in range(n):
        valuesYear = [startValue]
        for year in range(2022,2022+years):
            temp = random.choice(distirbutionDict["zufallsFaktor"]) * random.choice(distirbutionDict["birth"]) - random.choice(distirbutionDict["zufallsFaktor"]) * random.choice(distirbutionDict["death"]) + random.choice(distirbutionDict["zufallsFaktor"]) * random.choice(distirbutionDict["einwanderung"]) - random.choice(distirbutionDict["zufallsFaktor"]) * random.choice(distirbutionDict["auswanderung"])
            newValue = valuesYear[-1] + temp
            valuesYear.append(newValue)
        runs.append(np.array(valuesYear))

    return np.array(runs)

def MeanOfSim(runs:np.array):
    return runs.mean(axis=0)

n = 1000
Deutsch = simulation(distirbutionDict["Deutsch"],5243758,years=50,n=n)
Französisch = simulation(distirbutionDict["Französisch"],1922946,years=50,n=n)
Italienisch = simulation(distirbutionDict["Italienisch"],669870,years=50,n=n)
Andere = simulation(distirbutionDict["Andere"],2214643,years=50,n=n)

fig, ax = plt.subplots(4, 1, figsize=(15,15))

#plot Deutsch
for i in range(len(Deutsch)):
    ax[0].plot(Deutsch[i])

ax[0].plot(MeanOfSim(Deutsch), color="black", linewidth=5)

ax[0].set_ylim([0, 10000000])
ax[0].set_title("Deutsch")

for i in range(len(Französisch)):
    ax[1].plot(Französisch[i])

ax[1].plot(MeanOfSim(Französisch), color="black", linewidth=5)

ax[1].set_ylim([0, 10000000])
ax[1].set_title("Französisch")

for i in range(len(Italienisch)):
    ax[2].plot(Italienisch[i])

ax[2].plot(MeanOfSim(Italienisch), color="black", linewidth=5)

ax[2].set_ylim([0, 10000000])
ax[2].set_title("Italienisch")

for i in range(len(Andere)):
    ax[3].plot(Andere[i])

ax[3].plot(MeanOfSim(Andere), color="black", linewidth=5)

ax[3].set_ylim([0, 10000000])
ax[3].set_title("Andere")

plt.show()